## Notebook Description

This is "experimental" work to get the data transformation process down separate from the scraping logic.

In [22]:
import numpy as np
import pandas as pd
from transformer import process_all_dates
from datetime import datetime, timedelta

Generate fake data

In [29]:
dates = ['May 10, 2021', 'May 11, 2021'] # some fake dates
closing_prices = ['45.45', '48.64', '47.34', '44.89', '43.44', '70.49', '218.73', '438.24', '558.40', '552.43', '580.21', '506.76', '377.32', '168.27', '289.80', '325.83', '499.20', '627.39', '567.60', '484.54', '94.33', '65.78', '46.82', '44.73']
for x in range(len(closing_prices)):
    closing_prices[x] = float(closing_prices[x])

hour_values = [closing_prices, closing_prices]
    

fake_data = np.random.rand(24).tolist()

all_data = [[['9496', 'RETAILER', 'VQ6'], 
             ['9496', 'RETAILER', '941B']],
            [['9496', 'RETAILER', 'BTR1'], 
             ['9496', 'RETAILER', '941D']]]

all_data2 = [[['9496', 'RETAILER', 'VQ6'],
             ['9123', 'RETAILER', 'BTR1'],
             ['9496', 'RETAILER', '941B']],
            [['9496', 'RETAILER', 'BTR1'], 
             ['9496', 'RETAILER', '941D']]]

for d_idx, day_of_data in enumerate(all_data):
    for r_idx, row in enumerate(day_of_data):
        all_data[d_idx][r_idx].extend(fake_data)

for d_idx, day_of_data in enumerate(all_data):
    for r_idx, row in enumerate(day_of_data):
        all_data2[d_idx][r_idx].extend(fake_data)

In [30]:
result = process_all_dates(all_data, hour_values, dates)
result2 = process_all_dates(all_data2, hour_values, dates)

In [31]:
# result['Date'] = pd.to_datetime(result['Date'], format='%B %d, %Y')

In [32]:
result.head()

,Asset ID,Name,Date,Hour,MWh,Peak Status,Last Updated
0,VQ6,Waterton,2021-05-10,1,0.399981,Off-Peak,2021-06-08
1,VQ6,Waterton,2021-05-10,2,0.208514,Off-Peak,2021-06-08
2,VQ6,Waterton,2021-05-10,3,0.048453,Off-Peak,2021-06-08
3,VQ6,Waterton,2021-05-10,4,0.686919,Off-Peak,2021-06-08
4,VQ6,Waterton,2021-05-10,5,0.294268,Off-Peak,2021-06-08


In [5]:
result2["MWh"] = np.where((result2['Asset ID'] == 'VQ6') & (result2["Hour"]==2), 123, result2["MWh"])
result2

,Asset ID,Name,Date,Hour,MWh,Peak Status,Last Updated
0,BTR1,Blue Trail,"May 10, 2021",1,0.529617,Off-Peak,2021-06-08
1,BTR1,Blue Trail,"May 10, 2021",2,0.710821,Off-Peak,2021-06-08
2,BTR1,Blue Trail,"May 10, 2021",3,0.318073,Off-Peak,2021-06-08
3,BTR1,Blue Trail,"May 10, 2021",4,0.692961,Off-Peak,2021-06-08
4,BTR1,Blue Trail,"May 10, 2021",5,0.231121,Off-Peak,2021-06-08
...,...,...,...,...,...,...,...
67,BTR1,Blue Trail,"May 11, 2021",20,0.374686,Off-Peak,2021-06-08
68,BTR1,Blue Trail,"May 11, 2021",21,0.249106,Off-Peak,2021-06-08
69,BTR1,Blue Trail,"May 11, 2021",22,0.566290,Off-Peak,2021-06-08
70,BTR1,Blue Trail,"May 11, 2021",23,0.756489,Off-Peak,2021-06-08


In [12]:
result.set_index(['Asset ID', 'Date', 'Hour'], inplace=True)
result2.set_index(['Asset ID', 'Date', 'Hour'], inplace=True)

In [27]:

# merged = result2.merge(result, indicator=True, how='outer')
joined = result2.join(result['MWh'], how='left', rsuffix='_prev')
# merged['Last Updated'] = np.where(merged['_merge'] == 'left_only', 'Update', merged['Last Updated'])

joined['Last Updated'] = np.where(joined['MWh_prev'] != joined['MWh'], 
                          (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d'),
                          joined['Last Updated'])


pd.set_option('display.max_rows', 500)
joined

Name         MWh Peak Status Last Updated  \
Asset ID Date         Hour                                                    
BTR1     May 10, 2021 1     Blue Trail    0.529617    Off-Peak   2021-06-08   
                      2     Blue Trail    0.710821    Off-Peak   2021-06-08   
                      3     Blue Trail    0.318073    Off-Peak   2021-06-08   
                      4     Blue Trail    0.692961    Off-Peak   2021-06-08   
                      5     Blue Trail    0.231121    Off-Peak   2021-06-08   
                      6     Blue Trail    0.596446    Off-Peak   2021-06-08   
                      7     Blue Trail    0.868179    Off-Peak   2021-06-08   
                      8     Blue Trail    0.189386        Peak   2021-06-08   
                      9     Blue Trail    0.665962        Peak   2021-06-08   
                      10    Blue Trail    0.227317        Peak   2021-06-08   
                      11    Blue Trail    0.984214        Peak   2021-06-08   
                      12    Blue Trail    0.136436    Off-Peak   2021-06-08   
                      13    Blue Trail    0.632252    Off-Peak   2021-06-08   
                      14    Blue Trail    0.861081    Off-Peak   2021-06-08   
                      15    Blue Trail    0.508835    Off-Peak   2021-06-08   
                      16    Blue Trail    0.862117    Off-Peak   2021-06-08   
                      17    Blue Trail    0.249536    Off-Peak   2021-06-08   
                      18    Blue Trail    0.898809    Off-Peak   2021-06-08   
                      19    Blue Trail    0.322882    Off-Peak   2021-06-08   
                      20    Blue Trail    0.374686    Off-Peak   2021-06-08   
                      21    Blue Trail    0.249106    Off-Peak   2021-06-08   
                      22    Blue Trail    0.566290    Off-Peak   2021-06-08   
                      23    Blue Trail    0.756489    Off-Peak   2021-06-08   
                      24    Blue Trail    0.622222    Off-Peak   2021-06-08   
VQ6      May 10, 2021 1       Waterton    0.529617    Off-Peak   2021-06-08   
                      2       Waterton  123.000000    Off-Peak   2021-06-08   
                      3       Waterton    0.318073    Off-Peak   2021-06-08   
                      4       Waterton    0.692961    Off-Peak   2021-06-08   
                      5       Waterton    0.231121    Off-Peak   2021-06-08   
                      6       Waterton    0.596446    Off-Peak   2021-06-08   
                      7       Waterton    0.868179    Off-Peak   2021-06-08   
                      8       Waterton    0.189386        Peak   2021-06-08   
                      9       Waterton    0.665962        Peak   2021-06-08   
                      10      Waterton    0.227317        Peak   2021-06-08   
                      11      Waterton    0.984214        Peak   2021-06-08   
                      12      Waterton    0.136436    Off-Peak   2021-06-08   
                      13      Waterton    0.632252    Off-Peak   2021-06-08   
                      14      Waterton    0.861081    Off-Peak   2021-06-08   
                      15      Waterton    0.508835    Off-Peak   2021-06-08   
                      16      Waterton    0.862117    Off-Peak   2021-06-08   
                      17      Waterton    0.249536    Off-Peak   2021-06-08   
                      18      Waterton    0.898809    Off-Peak   2021-06-08   
                      19      Waterton    0.322882    Off-Peak   2021-06-08   
                      20      Waterton    0.374686    Off-Peak   2021-06-08   
                      21      Waterton    0.249106    Off-Peak   2021-06-08   
                      22      Waterton    0.566290    Off-Peak   2021-06-08   
                      23      Waterton    0.756489    Off-Peak   2021-06-08   
                      24      Waterton    0.622222    Off-Peak   2021-06-08   
BTR1     May 11, 2021 1     Blue Trail    0.529617    Off-Peak   2021-06-08   
     

In [8]:
result2

,Asset ID,Name,Date,Hour,MWh,Peak Status,Last Updated
0,VQ6,Waterton,"May 10, 2021",1,0.529617,Off-Peak,2021-06-08
1,VQ6,Waterton,"May 10, 2021",2,0.710821,Off-Peak,2021-06-08
2,VQ6,Waterton,"May 10, 2021",3,0.318073,Off-Peak,2021-06-08
3,VQ6,Waterton,"May 10, 2021",4,0.692961,Off-Peak,2021-06-08
4,VQ6,Waterton,"May 10, 2021",5,0.231121,Off-Peak,2021-06-08
5,VQ6,Waterton,"May 10, 2021",6,0.596446,Off-Peak,2021-06-08
6,VQ6,Waterton,"May 10, 2021",7,0.868179,Off-Peak,2021-06-08
7,VQ6,Waterton,"May 10, 2021",8,0.189386,Peak,2021-06-08
8,VQ6,Waterton,"May 10, 2021",9,0.665962,Peak,2021-06-08
9,VQ6,Waterton,"May 10, 2021",10,0.227317,Peak,2021-06-08


In [9]:
pd.concat([result, result2['MWh']], axis=1)

,Asset ID,Name,Date,Hour,MWh,Peak Status,Last Updated,MWh
0,VQ6,Waterton,"May 10, 2021",1.0,0.529617,Off-Peak,2021-06-08,0.529617
1,VQ6,Waterton,"May 10, 2021",2.0,0.710821,Off-Peak,2021-06-08,0.710821
2,VQ6,Waterton,"May 10, 2021",3.0,0.318073,Off-Peak,2021-06-08,0.318073
3,VQ6,Waterton,"May 10, 2021",4.0,0.692961,Off-Peak,2021-06-08,0.692961
4,VQ6,Waterton,"May 10, 2021",5.0,0.231121,Off-Peak,2021-06-08,0.231121
5,VQ6,Waterton,"May 10, 2021",6.0,0.596446,Off-Peak,2021-06-08,0.596446
6,VQ6,Waterton,"May 10, 2021",7.0,0.868179,Off-Peak,2021-06-08,0.868179
7,VQ6,Waterton,"May 10, 2021",8.0,0.189386,Peak,2021-06-08,0.189386
8,VQ6,Waterton,"May 10, 2021",9.0,0.665962,Peak,2021-06-08,0.665962
9,VQ6,Waterton,"May 10, 2021",10.0,0.227317,Peak,2021-06-08,0.227317
